## ПолныйПоиск

In [1]:
def auth(): # Авторизация + скачивание данных
    gauth = GoogleAuth()
    gauth.LocalWebserverAuth()
    drive = GoogleDrive(gauth)
    
    file = drive.ListFile({'q': "title = 'Nice2MeetYou' "}).GetList()[0] # Онлайн кабинет
    file.GetContentFile("Ввод/" + file['title'] + '.xlsx', mimetype = 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet')
    global fileNice2
    fileNice2 = "Ввод/" + file['title'] + '.xlsx'    
    print('Файл скачан: {}.'.format(file['title'] + '.xlsx'))
    
    folder_root_id = drive.ListFile({'q': "title = 'ПоискПолныйТГ' "}).GetList()[0]['id'] # Telegram каналы
    global filesTelegram
    filesTelegram = []
    for file in drive.ListFile({'q': " '%s' in parents" % (folder_root_id)}).GetList():
        file.GetContentFile("Ввод/" + file['title'])
        filesTelegram.append("Ввод/" + file['title'])
        print('Файл скачан: {}.'.format(file['title']))
        
    folder_root_id = drive.ListFile({'q': "title = 'ПоискПолныйИгры' "}).GetList()[0]['id'] # Построй компанию / продай компанию
    global filesPKPK
    filesPKPK = []
    for file in drive.ListFile({'q': "title contains '.html' and trashed = false and '%s' in parents" % (folder_root_id)}).GetList():
        file.GetContentFile("Ввод/" + file['title'])
        filesPKPK.append("Ввод/" + file['title'])
        print('Файл скачан: {}.'.format(file['title']))

In [2]:
def get_names(df): # Все имена из таблицы
    names = df[(df['type'] == 'service') & (df['action'] == 'invite_members')].explode('members')['members'].unique()
    names = [name for name in names if name and re.match("^[a-zA-Zа-яёА-ЯЁ0-9_. ]*$", name)]
    return names

In [3]:
def get_messages(df, name): # Все сообщения Telegram
    messages = df[(df['type'] == 'message') & (df['from'] == name)]['text']
    return messages

In [4]:
def transliterate(string): # Транслитерация
    let_dict = {u'а': u'a', u'б': u'b', u'в': u'v', u'г': u'g', u'д': u'd', u'е': u'e', u'ё': u'e', u'ж': u'zh', u'з': u'z',
                    u'и': u'i', u'й': u'y', u'к': u'k', u'л': u'l', u'м': u'm', u'н': u'n', u'о': u'o', u'п': u'p', u'р': u'r',
                    u'с': u's', u'т': u't', u'у': u'u', u'ф': u'f', u'х': u'h', u'ц': u'ts', u'ч': u'ch', u'ш': u'sh',
                    u'щ': u'sch', u'ъ': u'', u'ы': u'y', u'ь': u'', u'э': u'e', u'ю': u'yu', u'я': u'ya',} 
    result = u""
    for index, char in enumerate(string.lower()):
        if char in let_dict:
            char = let_dict[char]
        result += char
    return result.title()

In [5]:
def in_df(df, name): # Имя уже есть
    result = False
    for name1 in df.index.to_list():
        if ' ' in name and transliterate(name[:name.find(' ')]) in transliterate(name1).split() and transliterate(name[1+name.find(' '):]) in transliterate(name1).split():
            result = name1
            break
    return(result)

In [6]:
def date_format(string): # Форматирование даты
    ymd = string.split(' ')[0].split('-')
    return ymd[2] + '.' + ymd[1] + '.' + ymd[0]

In [7]:
def get_names_complete(file, Tfiles, Pfiles): # Все имена
    df0 = pd.read_excel(file, dtype = str, keep_default_na = False)
    Nnames = df0['name'].to_list()

    Tnames = []
    for file in Tfiles:
        data = pd.read_json(file, encoding = 'utf-8')
        df1 = pd.DataFrame.from_records(data['messages'])
        Tnames += list(get_names(df1))
        Tnames = list(dict.fromkeys(Tnames))

    Pnames = []
    for file in Pfiles:
        data_list = pd.read_html(file,encoding = 'utf-8')
        df2 = [data for data in data_list if 'Имя' in data.columns][0]
        Pnames += df2['Имя'].to_list()
        Pnames = list(dict.fromkeys(Pnames))
    
    global df # Таблица всех имен
    df = pd.DataFrame({'Имя': list(dict.fromkeys(Nnames))})
    df['Город проживания'] = df0['gorodprojivaniya']
    df['Онлайн-кабинет'] = 1
    df['Дата о-к'] = df0['sended'].apply(date_format)
    df['Telegram каналов'] = 0
    df['Дата Т'] = 'нет'
    df['Сообщений Т'] = 0
    df['Игры ПКПК'] = 0
    df['Дата П'] = 'нет'
    df['Баланс П'] = 'нет'
    df.set_index('Имя', inplace=True)
    
    for name in Tnames:
        d_count, c_count, m_count = telegram_count(Tfiles, name)
        name_new = in_df(df, name)
        if name_new:
            if df.at[name_new, 'Дата Т'] == 'нет':
                df.at[name_new, 'Дата Т'] = d_count
            else:
                df.at[name_new, 'Дата Т'] += d_count
            df.at[name_new, 'Telegram каналов'] += c_count
            df.at[name_new, 'Сообщений Т'] += m_count
        else:
            row_default = {'Город проживания': 'не указан', 'Онлайн-кабинет': 0, 'Дата о-к': 'нет', 'Telegram каналов': 0, 'Дата Т': 'нет', 'Сообщений Т': 0, 'Игры ПКПК': 0, 'Дата П': 'нет', 'Баланс П': 'нет'}
            row_default.update({'Дата Т': d_count, 'Telegram каналов': c_count, 'Сообщений Т': m_count})
            df.loc[name] = row_default
   
    for name in Pnames:
        d_count, g_count, b_count = pkpk_count(Pfiles, name)
        name_new = in_df(df, name)
        if name_new:
            if df.at[name_new, 'Дата П'] == 'нет':
                df.at[name_new, 'Дата П'] = d_count
                df.at[name_new, 'Баланс П'] = b_count
            else:
                df.at[name_new, 'Дата П'] += d_count
                df.at[name_new, 'Баланс П'] += b_count
            df.at[name_new, 'Игры ПКПК'] += g_count
        else:
            row_default = {'Город проживания': 'не указан', 'Онлайн-кабинет': 0, 'Дата о-к': 'нет', 'Telegram каналов': 0, 'Дата Т': 'нет', 'Сообщений Т': 0, 'Игры ПКПК': 0, 'Дата П': 'нет', 'Баланс П': 'нет'}
            row_default.update({'Дата П': d_count, 'Игры ПКПК': g_count, 'Баланс П': b_count})
            df.loc[name] = row_default
            
    return(df)

In [8]:
def telegram_count(files, name): # Счётчик каналов и сообщений
    d_count = []
    c_count = 0
    m_count = 0
    for file in files:
        data = pd.read_json(file, encoding = 'utf-8')
        df = pd.DataFrame.from_records(data['messages'])
        if name in get_names(df):
            d_count.append(file[5:-5])
            c_count += 1
            m_count += len(get_messages(df, name).to_list())
    return (d_count, c_count, m_count)

In [9]:
def pkpk_count(files, name): # Счётчик игр и баланса
    d_count = []
    g_count = 0
    b_count = []
    for file in files:
        data_list = pd.read_html(file,encoding = 'utf-8')
        df = [data for data in data_list if 'Имя' in data.columns][0]
        if name in df['Имя'].to_list():
            d_count.append(file[5:-12])
            g_count += 1
            b_count.append(int(df[df['Имя'] == name]['Баланс'].values[0]))
    return (d_count, g_count, b_count)

In [10]:
def search(names, name1, name2, trans = False): # Поиск по Ф и И
    if trans:
        r = regex.compile(r"(%s){e<=1}" % transliterate(name1.lower()))
        r1 = regex.compile(r"(%s){e<=1}" % transliterate(name2.lower()))
        result = [name for name in names if r.findall(transliterate(name.lower())) and r1.findall(transliterate(name.lower()))]
    else:
        r = regex.compile(r"(%s){e<=1}" % name1.lower())
        r1 = regex.compile(r"(%s){e<=1}" % name2.lower())
        result = [name for name in names if r.findall(name.lower()) and r1.findall(name.lower())]
    return result

In [11]:
def list_sort(series, method): # Сортировка
    result = list(series)
    if method == 'алфавитная':
        result.sort()
    elif method == 'алфавитная обратная':
        result.sort(reverse = True)
    return result

In [12]:
def list_max(list1):
    result = [x[x.find(' ')+1:] for x in list1]
    result = max(result, key=result.count)
    return result

In [13]:
def create_report(name): # ВОПРОС потом
    doc = docx.Document()
    doc.add_heading(name+". Результат новеллы.", 0)
    for file in files:
        data = pd.read_json(file, encoding = 'utf-8')
        df = pd.DataFrame.from_records(data['messages'])
        doc.add_paragraph(get_messages(df, name).to_list())
    doc.save("Вывод/{}.результат новеллы.docx".format(name))
    print('Файл создан: Вывод/{}.результат новеллы.docx'.format(name))

In [14]:
def fast_report(df, name, N, P, T): # Быстрый отчет
    report = df.loc[name]
    print("Быстрый отчет\n" + name + "\nГород проживания: " + report['Город проживания']  + "\n--------------------------------------------")
    if N:
        print("Онлайн-кабинет прохождения: "+ str(report['Онлайн-кабинет']) + 
            "\nДата прохождения: " + report['Дата о-к'] + "\n--------------------------------------------")
    if P:
        print("ИгрыПКПК: " + str(report['Игры ПКПК']) +
            "\nДата и баланс: " + str(report['Баланс П']) + "\n--------------------------------------------")
    if T:
        print("Telegram каналов: " + str(report['Telegram каналов']) +
            "\nСообщений: " + str(report['Сообщений Т']) + "\n--------------------------------------------")

In [15]:
#transliterate('Хущельпеков Гужельбег'.lower())

## Тестик

In [16]:
def make_window1(settings):
    tab1_layout = [[sg.Button('Загрузить', size=(5, 2))]]
    
    tab2_layout= [*[[sg.Frame('Условие {}'.format(i),
                              [[sg.Combo(list(if_dict.keys()), key='-IF{}-'.format(i),  enable_events=True, readonly=True),
                                sg.Input('10', size=(10, 1), key='-INT{}-'.format(i), enable_events=True, visible=False, tooltip='Число'),
                                sg.Combo([], size=(40, 10), key='-STR{}-'.format(i), enable_events=True, visible=False, tooltip='Список', readonly=True),
                                sg.Input('01.01.2021', size=(10,1), key='-DATE{}-'.format(i), enable_events=True, visible=False, tooltip='Дата')
                               ]])] for i in range(1, 6)],
                            [sg.Button('Поиск по условиям'), sg.Button('Очистить условия', key='-CLEAR-')]]
    
    tab3_layout = [[sg.Frame('Фамилия', [[sg.InputText('', key='-LAST-')]])],
                            [sg.Frame('Имя', [[sg.InputText('', key='-NAME-')]])],
                            [sg.Button('Поиск по имени')]]
    
    tab4_layout = [[sg.Frame('Сортировка', 
                            [[sg.Frame('Колонка', [*[[sg.Radio(opt, 'sort', default=(i==0), key='-SORT{}-'.format(i))] for i, opt in enumerate(sort_options)]]),
                             sg.Column(
                            [[sg.Frame('Порядок', [*[[sg.Radio(opt, 'order', default=(i==0), key='-ORDER{}-'.format(i))] for i, opt in enumerate(order_options)]])],
                             [sg.Frame('Формат таблицы',
                              [[sg.Radio('Компактный', 'format', default=True, key='-FORMAT_PART-', enable_events=True, tooltip='Сокращенные названия колонок, комбинированный формат, оптимизированный вид')],
                               [sg.Radio('Развернутый', 'format', default=False, key='-FORMAT_FULL-', enable_events=True, tooltip='Полные названия колонок, одиночный формат, автоматический вид')]])]])]])]]
    
    tab5_layout = [[sg.Frame('Внешний вид', 
                            [[sg.Column(
                                [[sg.Frame('Тема', [[sg.Combo(list(themes.keys()), default_value=sg.theme(), key='-THEME-', enable_events=True, readonly=True)]])],
                                 [sg.Frame('Цвет строк', [*[[sg.Radio(opt, 'color_alt', default=(i==0), key='-ALT_ROW_COLOR{}-'.format(i), circle_color=opt, background_color=opt, enable_events=True)] for i, opt in enumerate(themes[sg.theme()])]])]]),
                              sg.Column(
                                [[sg.Frame('Шрифт', [[sg.Combo(fonts, default_value=settings['font'], key='-FONT-', enable_events=True, readonly=True, tooltip=tooltip_fonts_emphasis)]])],
                                 [sg.Frame('Размер', [[sg.Combo([10, 12], default_value=settings['font_size'], key='-FONT_SIZE-', enable_events=True, readonly=True)]])],
                                 #[sg.Frame('Цвет', [*[[sg.Radio(opt, 'color_alt', default=(i==0), key='-ALT_ROW_COLOR{}-'.format(i), enable_events=True)] for i, opt in enumerate(themes[sg.theme()])]])]
                                ])
                                    ]]),
                              sg.Table([['Фамилия Имя '+str(i+1), 'Город '+str(i+1), '0{}.01.2021'.format(i+1), 'Data '+str(i+1), '0{}.01.2021 Город {}'.format(i+1, i+1)] for i in range(5)],
                                        headings=['Имя', 'Город', 'Дата', 'Data', 'Данные'],
                                        num_rows=5,
                                        justification='left',
                                        hide_vertical_scroll=True,
                                        max_col_width=50,
                                        auto_size_columns=True,
                                        #text_color = 'yellow',
                                        select_mode=sg.TABLE_SELECT_MODE_BROWSE,
                                        alternating_row_color=themes[sg.theme()][0],
                                        key='-TABLE-'
                                        )]]

    tab_group_layout = [[sg.Tab('Поиск по условиям', tab2_layout, key='-TAB1-'),
                                      sg.Tab('Поиск по имени', tab3_layout, key='-TAB2-'),
                                      sg.Tab('Настройки', tab4_layout, key='-TAB3-'),
                                      sg.Tab('Вид', tab5_layout, key='-TAB4-')]]

    layout = [[sg.TabGroup(tab_group_layout,
                               enable_events=True,
                               key='-TABGROUP-')]]

    return sg.Window('ПоискПолный v0.02', layout, no_titlebar=False, finalize=True)

def make_window2():
    layout = [[sg.Text('Нет данных!')],
                   [sg.Button('OK')]]
    
    return sg.Window('Window 2', layout, finalize=True)

def make_window3(df, sort_key='По умолчанию', sort_ascending=True, alt_row_color=None, format_full=True):
    data_frame = df.rename(columns=rename_columns)
    if sort_key != 'По умолчанию':
        if sort_key in ['Имя', 'Город проживания']:
            data_frame.replace({'Город проживания': {'не указан': np.nan}}, inplace=True)
            data_frame.sort_values(by=sort_key, key=lambda col: col.str.lower(), ascending=sort_ascending, inplace=True)
            data_frame.replace({'Город проживания': {np.nan: 'не указан'}}, inplace=True)
        else:
            data_frame.sort_values(by=sort_key, ascending=sort_ascending, inplace=True)
    data_frame.replace({np.nan: 'нет'}, inplace=True)
    if format_full:
        data_frame.drop(data_frame.columns.difference(include_columns['format_full']), axis=1, inplace=True)
    else:
        data_frame.rename(columns=rename_columns_format_part, inplace=True)
        data_frame.drop(data_frame.columns.difference(include_columns['format_part']), axis=1, inplace=True)
    data = data_frame.values.tolist()
    header_list = data_frame.columns.to_list()
    if data == []:
        layout = [[sg.Text('Пустая таблица!')],
                       [sg.Button('OK')]]
    else:
        layout = [[sg.Table(values=data,
                            justification='left',
                            headings=header_list,
                            col_widths=(None if format_full else col_widths_format_part),
                            auto_size_columns=format_full,
                            max_col_width=45,
                            #text_color = 'yellow',
                            num_rows=min(25, len(data)),
                            enable_events = True,
                            alternating_row_color=alt_row_color,
                            key='-TABLE_MAIN-',
                            tooltip=(None if format_full else tooltip_table)
                            )]]

    return sg.Window('Таблица, {} записей'.format(len(data)), layout, finalize=True)

def create_table(df, method = 'condition', conditions = None):
    if method == 'name':
        name, last = conditions
        names = search(list(df['Имя']), name, last, True)
        result = df[df['Имя'].isin(names)]
    return result

def check_input(value, limit, dot = False):
    
    return(value and (value[-1] not in ('0123456789.' if dot else '0123456789') or len(value) > limit))

In [17]:
import datetime as dt
tooltip_table = 'Обозначения: Город проживания* = город проживания не указан явно;\n\
                        Дата о-к, % = дата прохождения онлайн-кабинета, первичный процент;\n\
                        Telegram к, с = количество каналов, сообщений в Telegram;\n\
                        ПКПК(п) баланс[м,ср] = количество игр ПКПК, число роли предпринимателя, максимальный и средний балансы;'

tooltip_fonts_emphasis = 'С выделением кириллицы:\nAndalus 12\nEbrima 12\nEuphemia 12\nKalinga 12\nPapyrus 12\nRaavi 12\nRockwell 12\nVani 12'

fonts = ['Andalus', 'Arial', 'Calibri', 'Cambria', 'Candara', 'Century', 'Corbel', 'Courier', 'Dotum', 'Ebrima', 'Euphemia', 'Garamond', 'Georgia', 'Helvetica', 'Kalinga', 'Lato', 'Meiryo', 'Papyrus', 'Raavi', 'Rockwell', 'Sylfaen', 'Tahoma', 'Vani', 'Verdana']

themes = {'Reddit': ['azure', 'alice blue', 'pale turquoise', 'dark turquoise', 'cadetblue1', 'steelblue1', 'pale green', 'gold'],
          'LightBlue': ['azure', 'alice blue', 'pale turquoise', 'dark turquoise', 'cadetblue1', 'steelblue1', 'pale green', 'gold'],
          'LightBrown10': ['wheat1', 'lemonchiffon2', 'antiquewhite1', 'sandy brown', 'sienna1', 'darkgoldenrod1', 'lightgoldenrod1', 'goldenrod1'],
          'DarkAmber': ['grey5', 'grey15', 'grey25', 'dark slate gray', 'dodgerblue4', 'saddle brown', 'dark green', 'dark olive green']}

if_dict = {
            'Город проживания совпадает с': [str, True, 'Город проживания'],
            'Онлайн-кабинет прохождений больше': [int, True, 'Онлайн-кабинет'],
            'Онлайн-кабинет прохождений меньше': [int, False, 'Онлайн-кабинет'],
            'Онлайн-кабинет дата после': [dt.datetime.strptime, True, 'Дата о-к'],
            'Онлайн-кабинет дата до': [dt.datetime.strptime, False, 'Дата о-к'],
            'Онлайн-кабинет первичный % больше': [int, True, 'Первичный П'],
            'Онлайн-кабинет первичный % меньше': [int, False, 'Первичный П'],
            'Telegram каналов больше': [int, True, 'Telegram каналов'],
            'Telegram каналов меньше': [int, False, 'Telegram каналов'],
            'Telegram каналов дата содержит': [str, False, 'Дата Т'],
            'Telegram каналов сообщений больше': [int, True, 'Сообщений Т'],
            'Telegram каналов сообщений меньше': [int, False, 'Сообщений Т'],
            'Игры ПКПК количество больше': [int, True, 'Игры ПКПК'],
            'Игры ПКПК количество меньше': [int, False, 'Игры ПКПК'],
            'Игры ПКПК дата содержит': [str, False, 'Дата П'],
            'Игры ПКПК роль предприниматель больше': [int, True, 'Роль П'],
            'Игры ПКПК роль предприниматель меньше': [int, False, 'Роль П'],
            'Игры ПКПК баланс максимальный больше': [int, True, 'Баланс макс'],
            'Игры ПКПК баланс максимальный меньше': [int, False, 'Баланс макс'],
            'Игры ПКПК баланс средний больше': [int, True, 'Баланс средний'],
            'Игры ПКПК баланс средний меньше': [int, False, 'Баланс средний']
            }

rename_columns = {'Дата о-к': 'Дата прохождения', 'Первичный П': 'Первичный %', 'Telegram каналов': 'Каналов Telegram', 'Дата Т': 'Дата Telegram', 'Сообщений Т': 'Сообщений', 'Роль П': 'Роль предприниматель'}

rename_columns_format_part = {'Дата прохождения': 'Дата о-к'}

col_widths_format_part = [28, 16, 12, 10, 30, 16, 70]

include_columns = {'format_full': ['№', 'Имя', 'Город проживания', 'Онлайн-кабинет', 'Дата прохождения', 'Первичный %', 'Каналов Telegram', 'Дата Telegram',
                                                    'Сообщений Т', 'Игры ПКПК(п)', 'Дата и баланс', 'Баланс макс', 'Баланс средний'],
                               'format_part': ['Имя', 'Город проживания', 'Дата о-к, %', 'Telegram к,с', 'Дата Telegram', 'ПКПК(п) баланс[м,ср]', 'Дата и баланс']}

sort_options = ['По умолчанию', 'Имя', 'Город проживания', 'Дата прохождения', 'Первичный %', 'Каналов Telegram', 'Сообщений', 'Игры ПКПК', 'Роль предприниматель', 'Баланс макс', 'Баланс средний']

order_options = ['По умолчанию', 'По убыванию', 'По возрастанию']

In [18]:
import PySimpleGUI as sg
import pandas as pd
import numpy as np
import regex
import re
import requests
from statistics import mean
from itertools import count

def main():
    try:
        #df = pd.read_csv('CompleteSearch.csv', encoding = 'windows-1251')
        url = 'https://10education.ru/CompleteSearch/select.php'

        r = requests.get(url)
        first_list = r.text.split('<br />')
        index_pass = first_list[0].index('/html')
        first_list[0] = first_list[0][index_pass+10:]
        second_list = [string.split('|') for string in first_list]
        df = pd.DataFrame(second_list)
        columns_names = {0: 'Имя', 1: 'Город проживания', 2: 'Онлайн-кабинет', 3: 'Дата о-к', 4: 'Первичный П', 5: 'Telegram каналов', 6: 'Дата Т',
                         7: 'Сообщений Т', 8: 'Игры ПКПК', 9: 'Роль П', 10: 'Дата П', 11: 'Баланс П'}
        df = df.rename(columns=columns_names)
        df = df[:-1]
        df['Первичный П'] = df['Первичный П'].astype(float)
        for col in ['Онлайн-кабинет', 'Telegram каналов', 'Сообщений Т', 'Игры ПКПК']:
            df[col] = df[col].astype(int)
        
        df.insert(0,'№', [str(x) for x in df.reset_index().index.to_list()])
        df.replace(0, '2')
        df['№'] = '  ' + df['№']
        df['Роль П'] = df['Роль П'].str.split(', ')
        df['Дата П'] = df['Дата П'].apply(lambda x: re.split(r", (?=\d)", x))
        df['Город проживания'] = [list_max(b)+"*" if (a == 'не указан') and (b != ['нет']) else a for a, b in zip(df['Город проживания'], df['Дата П'])]
        df['Баланс П'] = df['Баланс П'].str.split(', ')
        df['Дата о-к'] = pd.to_datetime(df['Дата о-к'], format='%d.%m.%Y', errors='coerce').dt.date
        df['Дата и баланс'] = [', '.join([a[i]+' ({},П)'.format(b[i]) if c[i] == '1' else a[i]+' ({})'.format(b[i]) for i in range(len(a))]) if a[0] != 'нет' else 'нет' for a, b, c in zip(df['Дата П'], df['Баланс П'], df['Роль П'])]
        df['Баланс макс'] = [max(int(aa) for aa in a) if a[0] != 'нет' else 0 for a in df['Баланс П']]
        df['Баланс средний'] = [int(mean(int(aa) for aa in a)) if a[0] != 'нет' else 0 for a in df['Баланс П']]
        df['Роль П'] = df['Роль П'].apply(lambda x: x.count('1'))
        df.insert(df.columns.get_loc('Дата о-к'), 'Дата о-к, %', df['Дата о-к'].replace({np.nan: 'нет'}).astype(str)+df['Первичный П'].apply(lambda x: ' ({})'.format(round(x))))
        df.insert(df.columns.get_loc('Telegram каналов'), 'Telegram к,с', +df['Telegram каналов'].astype(str)+', '+df['Сообщений Т'].astype(str))
        df.insert(df.columns.get_loc('Игры ПКПК'), 'Игры ПКПК(п)', df['Игры ПКПК'].astype(str)+df['Роль П'].apply(lambda x: '({})'.format(x) if x>0 else ''))
        df.insert(df.columns.get_loc('Игры ПКПК(п)'), 'ПКПК(п) баланс[м,ср]', df['Игры ПКПК'].astype(str)+df['Роль П'].apply(lambda x: '({})'.format(x) if x>0 else '')+': ['+df['Баланс макс'].astype(str)+', '+df['Баланс средний'].astype(str)+']')
        header_list = df.columns.to_list()
        #print('Данные загружены')
        sg.theme('Reddit')
        alt_row_color = themes[sg.theme()][0]
        settings = {'font': 'Helvetica', 'font_size': 10}
        window1 = make_window1(settings)
        window2 = None
        window3 = None
    
        while True:
            window, event, values = sg.read_all_windows()
            #print(event, values)
            sg.SetOptions(auto_size_text=True)
            if (window, event) == (window1, None):
                break

            if (window, event) == (window1, '-THEME-'):
                sg.theme(values[event])
                window.close()
                window1 = make_window1(settings)
                window1['-TAB4-'].select()

            if (window, event) == (window1, '-FONT-'):
                settings['font'] = values[event]
                sg.SetOptions(font=(values[event], settings['font_size']))
                window.close()
                window1 = make_window1(settings)
                window1['-TAB4-'].select()

            if (window, event) == (window1, '-FONT_SIZE-'):
                settings['font_size'] = values[event]
                sg.SetOptions(font=(settings['font'], values[event]))
                window.close()
                window1 = make_window1(settings)
                window1['-TAB4-'].select()

            if (window == window1 and event in ['-ALT_ROW_COLOR{}-'.format(i) for i in range(len(themes[sg.theme()]))]):
                alt_row_color = themes[sg.theme()][int(event[-2])]
                window['-TABLE-'].update(alternating_row_color = alt_row_color)

            if (window == window1 and event in ['-IF1-', '-IF2-', '-IF3-', '-IF4-', '-IF5-']):
                for key in [k + event[-2:] for k in ['-INT', '-STR', '-DATE']]:
                    window[key].update(visible=False)
                if_type, if_greater, if_column = if_dict[values[event]]
                if if_type == int:
                    window['-INT' + event[-2:]].update(visible=True)
                elif if_type == str:
                    window['-STR' + event[-2:]].update(visible=True)
                    if if_greater:
                        window['-STR' + event[-2:]].update(values=[key + ' ({})'.format(value) for key, value in df[if_column].apply(lambda x: x[:-1] if "*" in x else x).value_counts().to_dict().items()])
                    elif if_column == 'Дата Т':
                        window['-STR' + event[-2:]].update(values=sorted(list(set(', '.join(filter(lambda x: x != 'нет', df[if_column])).split(', '))), key=lambda x: dt.datetime.strptime(x[:x.find(' ')], '%d.%m.%Y'), reverse=True))
                    else:
                        window['-STR' + event[-2:]].update(values=sorted(list(set(filter(lambda x: x != 'нет', df[if_column].sum()))), key=lambda x: dt.datetime.strptime(x[:x.find(' ')], '%d.%m.%y'), reverse=True))
                elif if_type == dt.datetime.strptime:
                    window['-DATE' + event[-2:]].update(visible=True)

            if (window == window1 and event in ['-STR{}-'.format(i) for i in range(1,6)] and values[event].find(' (') != -1):
                window[event].update(values[event][:values[event].find(' (')])

            if (window == window1 and event in ['-INT{}-'.format(i) for i in range(1,6)] and check_input(values[event], 4)):
                window[event].update(values[event][:-1])

            if (window == window1 and event in ['-DATE{}-'.format(i) for i in range(1,6)] and check_input(values[event], 10, dot = True)):
                window[event].update(values[event][:-1])

            if (window, event) == (window1, '-CLEAR-'):
                for key in [k.format(i) for k in ['-INT{}-', '-STR{}-', '-DATE{}-'] for i in range(1,6)]:
                    window[key].update(visible=False)
                for i in range(1,6):
                    window['-IF{}-'.format(i)].update('')

            if (window, event) == (window1, 'Поиск по условиям'):
                window1.hide()
                data = df.copy()
                for i in range(1, 6):
                    if values['-IF{}-'.format(i)] in if_dict.keys():
                        if_type, if_greater, if_column = if_dict[values['-IF{}-'.format(i)]]
                        try:
                            if if_type == int:
                                value = if_type(values['-INT{}-'.format(i)])
                                if if_greater:
                                    data = data[data[if_column] > value]
                                else:
                                    data = data[data[if_column] < value]
                            elif if_type == str:
                                value = if_type(values['-STR{}-'.format(i)])
                                if if_greater:
                                    data = data[data[if_column].isin([value, value+"*"])]
                                elif type(df[if_column][0]) == str:
                                    data = data[data[if_column].str.contains(value)]
                                else:
                                    data = data[data[if_column].apply(lambda x: value in x)]
                            elif if_type == dt.datetime.strptime:
                                value = if_type(values['-DATE{}-'.format(i)], '%d.%m.%Y').date()
                                if if_greater:
                                    data = data[data[if_column] > value]
                                else:
                                    data = data[data[if_column] < value]
                        except:
                            data = pd.DataFrame(columns = data.columns)

                sort_key = [sort_options[i] for i in range(len(sort_options)) if values['-SORT{}-'.format(i)]][0]
                sort_ascending = values['-ORDER2-']
                format_full = values['-FORMAT_FULL-']
                window3 = make_window3(data, sort_key, sort_ascending, alt_row_color, format_full)
        
            if (window, event) == (window1, 'Поиск по имени'):
                window1.hide()
                sort_key = [sort_options[i] for i in range(len(sort_options)) if values['-SORT{}-'.format(i)]][0]
                sort_ascending = values['-ORDER2-']
                format_full = values['-FORMAT_FULL-']
                window3 = make_window3(create_table(df, 'name', [values['-LAST-'], values['-NAME-']]), sort_key, sort_ascending, alt_row_color, format_full)

            if (window == window3 and event in [None, 'OK']):
                window3.close()
                window3 = None
                window1.un_hide()
                
        window1.close()
    
    except Exception as e:
        print(e)
    #except:
     #   sg.popup('Данные не найдены!')
        
if __name__ == '__main__':
    main()